In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf

In [3]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)

In [4]:
def trend(time, slope=0):
    return slope * time

In [5]:
def seasonal_pattern(season_time):
    return np.where(season_time < 0.4,
    np.cos(season_time * 2 * np.pi),
    1 / np.exp(3 * season_time))

In [6]:
def seasonality(time, period, amplitude=1, phase=0):
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)

In [7]:
def noise(time, noise_level=1, seed = None):
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * noise_level

In [8]:
time = np.arange(4 * 365 + 1, dtype="float32")
baseline = 10
series = trend(time, .05)
amplitude = 15
slope = 0.09
noise_level = 6

In [9]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series,axis=-1)
    dataset =  tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    # 반환 값이 dataset
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    # [0 1 2 3 4] 형식으로 바로 받기 위해서 flat_map함수를 사용한다.
    dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    # prefetch 훈련속도에 영향을 준다. 하나의 데이터셋이 작업을 하는 동안 미리 다른 데이터셋을 준비
    return dataset

In [10]:
series = baseline + trend(time, slope) \
                  + seasonality(time, period=365, amplitude=amplitude)

In [11]:
split_time = 1000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]
window_size = 20
batch_size = 32
shuffle_buffer_size = 1000
dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [18]:
dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=128, kernel_size=3,
                           strides=1, padding="causal",
                           activation="relu",
                           input_shape=[None, 1]),
    tf.keras.layers.Dense(28, activation="relu"),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1),
])

optimizer = tf.keras.optimizers.SGD(lr=1e-5, momentum=0.5)
model.compile(loss="mse", optimizer=optimizer)
history = model.fit(dataset, epochs=100, verbose=1)


Epoch 1/100
31/31 [==============================] - 2s 6ms/step - loss: nan
Epoch 2/100
31/31 [==============================] - 0s 6ms/step - loss: nan
Epoch 3/100
31/31 [==============================] - 0s 5ms/step - loss: nan
Epoch 4/100
31/31 [==============================] - 0s 4ms/step - loss: nan
Epoch 5/100
31/31 [==============================] - 0s 4ms/step - loss: nan
Epoch 6/100
31/31 [==============================] - 0s 5ms/step - loss: nan
Epoch 7/100
31/31 [==============================] - 0s 4ms/step - loss: nan
Epoch 8/100
31/31 [==============================] - 0s 6ms/step - loss: nan
Epoch 9/100
31/31 [==============================] - 0s 5ms/step - loss: nan
Epoch 10/100
31/31 [==============================] - 0s 4ms/step - loss: nan
Epoch 11/100
31/31 [==============================] - 0s 4ms/step - loss: nan
Epoch 12/100
31/31 [==============================] - 0s 4ms/step - loss: nan
Epoch 13/100
31/31 [==============================] - 0s 5ms/step - loss:

In [19]:
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

In [20]:
forecast = model_forecast(model, series[..., np.newaxis], window_size)

46/46 [==============================] - 1s 6ms/step


In [21]:
results = forecast[split_time - window_size:-1, -1, 0]